# Self Driving Car Assignment Using keras



### Loading Dataset

In [1]:
from __future__ import division

import os
import numpy as np
import random
from scipy import pi
import imageio
import cv2
from itertools import islice


LIMIT = None

DATA_FOLDER = 'driving_dataset'
TRAIN_FILE = os.path.join(DATA_FOLDER, 'data.txt')

# the expected dimension is (tf ordering)
# (None, 66, 200, 3)
def return_data(split=.7):

    X = []
    y = []

    with open(TRAIN_FILE) as fp:
        for line in islice(fp, LIMIT):
            path, angle = line.strip().split()
            full_path = os.path.join(DATA_FOLDER, path)
            X.append(full_path)
            # using angles from -pi to pi to avoid rescaling the atan in the network
            y.append(float(angle) * pi / 180 )
    y = np.array(y)


    #images = np.array([np.float32(imageio.imread(im))/255  for im in X])
    images = np.array([np.float32(cv2.resize(imageio.imread(im,pilmode="RGB")[-150:], dsize=(200,66)))/255  for im in X])
    split_index = int(split * len(X))

    train_X = images[:split_index]
    train_y = y[:split_index]
    test_X = images[split_index:]
    test_y = y[split_index:]

    return np.array(train_X), np.array(train_y), np.array(test_X), np.array(test_y)


## Model Arcitecture

In [2]:

from keras.layers import Convolution2D, Input
from keras.layers.core import Dense, Flatten, Lambda, Activation, Dropout
from keras.models import Model, Sequential
from keras.optimizers import SGD
import keras
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, RemoteMonitor, EarlyStopping
from keras.layers import Conv2D,MaxPooling2D

import tensorflow as tf
"""
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
sess = InteractiveSession(config=config)"""

from keras import backend as K
K.common.image_dim_ordering()


# fina NVIDIA Architecture Model
# reference: https://github.com/hdmetor/Nvidia-SelfDriving

def NVIDA( lr_rate, d_rate, verbose=True ):

    
    nvidia_model= Sequential()
    nvidia_model.add(Convolution2D(24, kernel_size=(5, 5),padding='valid', activation='relu', strides=(2, 2), name='conv_1',input_shape=(66,200, 3) ))
    nvidia_model.add(Convolution2D(36, kernel_size=(5, 5),padding='valid', activation='relu', strides=(2, 2), name='conv_2' ))
    nvidia_model.add(Convolution2D(48, kernel_size=(5, 5),padding='valid', activation='relu', strides=(2, 2), name='conv_3' ))
    nvidia_model.add(Convolution2D(64, kernel_size=(3, 3),padding='valid', activation='relu', strides=(1, 1), name='conv_4'))
    nvidia_model.add(Convolution2D(64, kernel_size=(3, 3),padding='valid', activation='relu', strides=(1, 1), name='conv_5'))
    
    nvidia_model.add(Flatten())
    
    nvidia_model.add(Dense(1164, activation="relu"))
    nvidia_model.add(Dropout(rate=d_rate))
    
    nvidia_model.add(Dense(100, activation="relu"))
    nvidia_model.add(Dropout(rate=d_rate))
    
    nvidia_model.add(Dense(50, activation="relu"))
    nvidia_model.add(Dropout(rate=d_rate))
    
    nvidia_model.add(Dense(10, activation="relu"))
    nvidia_model.add(Dropout(rate=d_rate))
    
    nvidia_model.add(Dense(1))
    nvidia_model.add(Lambda(lambda x: keras.activations.tanh(x)))

    #optimiser inisilisation
    ada=keras.optimizers.adam(lr_rate)
    
    nvidia_model.compile(optimizer=ada,loss='mse')
    if verbose:
        nvidia_model.summary()
        
    return nvidia_model


Using TensorFlow backend.


'tf'

## Loading and Traning and Hyperparameter Tuning

In [4]:
import time
import json
import numpy as np
import pickle

print('Loading data...')
train_x, train_y, test_x, test_y = return_data(split=0.7)
print('Done')


Loading data...
Done


In [5]:
print(f"train shape: {train_x.shape}")
print(f"test shape: {test_x.shape}")

train shape: (31784, 66, 200, 3)
test shape: (13622, 66, 200, 3)


In [20]:
# Hyperparameter Tuning

#checkpoint callback
checkpointer = ModelCheckpoint(
    filepath="{epoch:02d}-{val_loss:.12f}.hdf5",
    verbose=1,
    save_best_only=True)

# Early Stopper callback
early_stopper = EarlyStopping(monitor='val_loss',min_delta=1e-4,patience=10,verbose=0,
                              mode='auto',baseline=None,restore_best_weights=True)

# reduce_learning_rate plateau callback
lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.000001, verbose=0, mode=min)


def SDC(train_x, train_y, test_x, test_y, params):

    nvidia_model= Sequential()
    nvidia_model.add(Convolution2D(24, kernel_size=(5, 5),padding='valid', activation='relu', strides=(2, 2), name='conv_1',input_shape=(66,200, 3) ))
    nvidia_model.add(Convolution2D(36, kernel_size=(5, 5),padding='valid', activation='relu', strides=(2, 2), name='conv_2' ))
    nvidia_model.add(Convolution2D(48, kernel_size=(5, 5),padding='valid', activation='relu', strides=(2, 2), name='conv_3' ))
    nvidia_model.add(Convolution2D(64, kernel_size=(3, 3),padding='valid', activation='relu', strides=(1, 1), name='conv_4'))
    nvidia_model.add(Convolution2D(64, kernel_size=(3, 3),padding='valid', activation='relu', strides=(1, 1), name='conv_5'))
    
    nvidia_model.add(Flatten())
    
    nvidia_model.add(Dense(1164, activation="relu"))
    nvidia_model.add(Dropout(rate=params["d_rate"]))
    
    nvidia_model.add(Dense(100, activation="relu"))
    nvidia_model.add(Dropout(rate=params["d_rate"]))
    
    nvidia_model.add(Dense(50, activation="relu"))
    nvidia_model.add(Dropout(rate=params["d_rate"]))
    
    nvidia_model.add(Dense(10, activation="relu"))
    nvidia_model.add(Dropout(rate=params["d_rate"]))
    
    nvidia_model.add(Dense(1))
    nvidia_model.add(Lambda(lambda x: keras.activations.tanh(x)))

    #optimiser inisilisation
    ada=keras.optimizers.adam(learning_rate=params["lr"])
    
    nvidia_model.compile(optimizer=ada,loss='mse')

    history = nvidia_model.fit(train_x, train_y, 
                        validation_data=[test_x, test_y],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0,
                        callbacks=[early_stopper,lr_plateau])
    
    # finally we have to make sure that history object and model are returned
    return history, nvidia_model

In [21]:
# making dict of hyperparameters
p = {'batch_size': (25,50,100,125),
     'epochs': [100],
     'd_rate':[0.5],
     "lr":[0.001,0.0001]}

p

{'batch_size': (25, 50, 100, 125),
 'epochs': [100],
 'd_rate': [0.5],
 'lr': [0.001, 0.0001]}

In [22]:
# hypertuning parameter using takos

import talos

t = talos.Scan(x=train_x,
            y=train_y,
            model=SDC,
            params=p,
            experiment_name='sdc_fnal')






  0%|                                                                                           | 0/50 [00:00<?, ?it/s]




  2%|█▌                                                                              | 1/50 [01:45<1:25:50, 105.11s/it]




  4%|███▏                                                                            | 2/50 [09:41<2:53:10, 216.46s/it]




  6%|████▊                                                                           | 3/50 [11:21<2:22:18, 181.68s/it]




  8%|██████▍                                                                         | 4/50 [22:28<4:10:51, 327.21s/it]




 10%|████████                                                                        | 5/50 [24:16<3:16:02, 261.39s/it]




 12%|█████████▌                                                                      | 6/50 [33:46<4:19:37, 354.04s/it]




 14%|███████████▏                                                                    | 7/50 [35:26<3:18:57, 277.62s/it]




 16

In [27]:
result = t.data
result.head()

,round_epochs,val_loss,loss,lr,batch_size,d_rate,epochs,lr
0,11,0.319554,0.309572,0.0010,25,0.5,100,0.0010
1,52,0.122518,0.127042,0.0001,25,0.5,100,0.0001
2,11,0.319588,0.309542,0.0010,26,0.5,100,0.0010
3,74,0.121853,0.121453,0.0001,26,0.5,100,0.0001
4,12,0.319623,0.309552,0.0010,27,0.5,100,0.0010


In [26]:
# optimal hyperparameter
result[result.val_loss==result.val_loss.min()]

,round_epochs,val_loss,loss,lr,batch_size,d_rate,epochs,lr
39,87,0.121049,0.125895,0.0001,44,0.5,100,0.0001


#### Traning using optimal hyper parameter

In [28]:
# Traning using optimal hyper parameter

#checkpoint callback
checkpointer = ModelCheckpoint(
    filepath="{epoch:02d}-{val_loss:.12f}.hdf5",
    verbose=1,
    save_best_only=True)

# Early Stopper callback
early_stopper = EarlyStopping(monitor='val_loss',min_delta=1e-4,patience=10,verbose=2,
                              mode='auto',baseline=None,restore_best_weights=True)

# reduce_learning_rate plateau callback
lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.000001, verbose=1, mode=min)


# optimal hyperparameter
epochs = 100
batch_size = 44
lr=0.0001
d=0.5

print('Loading model')
nvidia = NVIDA(lr_rate=lr, d_rate=d)
print('Starting training')
history = nvidia.fit(train_x, train_y,
                     validation_data=(test_x, test_y),
                     nb_epoch=epochs,
                     batch_size=batch_size,
                     verbose=2,
                     callbacks=[checkpointer])


with open( f'history_{time.time()}.pkl' , 'wb') as fp:
    # dict with np array as items are not serializable
    pickle.dump(dict((k, np.array(v).tolist()) for k, v in history.history.items()), fp)
print('Done')


Loading model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
conv_2 (Conv2D)              (None, 14, 47, 36)        21636     
_________________________________________________________________
conv_3 (Conv2D)              (None, 5, 22, 48)         43248     
_________________________________________________________________
conv_4 (Conv2D)              (None, 3, 20, 64)         27712     
_________________________________________________________________
conv_5 (Conv2D)              (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1164

In [31]:
# Loading best model checkpoint

nvidia = NVIDA(lr_rate=lr, d_rate=d, verbose=0)

nvidia.load_weights("41-0.148576635343.hdf5")
scores = nvidia.evaluate(test_x,test_y,verbose=0)
print( f"{nvidia.metrics_names}: {scores}")

['loss']: 0.14857663764049028


In [32]:
pred_angle = nvidia.predict(test_x)
pred_angle[:15]

array([[-0.39942533],
       [-0.36128697],
       [-0.32552075],
       [-0.2734226 ],
       [-0.19971263],
       [-0.14588314],
       [-0.11527184],
       [-0.01421635],
       [-0.04970898],
       [ 0.06233919],
       [ 0.09712008],
       [ 0.07112201],
       [-0.16278084],
       [-0.20308086],
       [-0.21070103]], dtype=float32)

### visualisation of predicted angle

In [39]:
# visualisation predicted angle result
#pip3 install opencv-python

import scipy.misc
import cv2
from subprocess import call
import math
import imageio
import time


img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))

while(cv2.waitKey(10) != ord('q')):
    full_image = imageio.imread("driving_dataset/" + str(i) + ".jpg", pilmode="RGB")
    image = cv2.resize(full_image[-150:], dsize=(200,66)) / 255.0
    
    degrees = (nvidia.predict(image.reshape(1,66,200,3))[0][0]* 180.0 )/scipy.pi  
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    
    #print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    #print("smoothed_angle: ",smoothed_angle)
    
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    time.sleep(0.02)
    i += 1
    
cv2.destroyAllWindows()


Starting frameofvideo:36325


# Observation

1. Although traning data was not very large still Nvidia end to end model is doing quite well in prediction angle of stering wheels (ofcourse not near close to use in real life).


END :)